# EPM Opened Processes Scrapper

This scrapper get the code of the opened procceses in the EPM webpage

## Import required libraries

In [2]:
import re
import time
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By


## Start process

### Start chrome browser

In [3]:
driver = webdriver.Chrome()


### Turn chrome in Maximize windows

In [4]:
driver.maximize_window()

### Get EPM webpage

In [5]:
driver.get('https://portalesepm.epm.com.co/TeCuento/ProcesosComprasMenoresVigentes/ConsultarProcesosCompras.aspx')

### Accept conditions and into processes page

Find accept button

In [6]:
aceptar= driver.find_element(by=By.XPATH, value='''//*[@id="ctl00_ContentPlaceHolderContenido_ChkCondicionesCOntractuales"]''')

If find the button, click on it

In [7]:
aceptar.click()

Click into accept button in emergent window

In [8]:
driver.find_element(by=By.XPATH, value='''//*[@id="ctl00_ContentPlaceHolderContenido_btnAceptar"]''').click()

Wait until the page's load

In [9]:
time.sleep(1)

### Get the processes table

Click on query button

In [10]:
driver.find_element(by=By.XPATH, value='''//*[@id="ctl00_ContentPlaceHolderContenido_btnConsultar"]''').click()

Wait until the table load

In [11]:
time.sleep(5)

### Get the page numbers of the table

Find the pages table

In [12]:
numero_pagina = driver.find_elements(by=By.XPATH, value='''//*[@id="ctl00_ContentPlaceHolderContenido_gvListadoProcesosNoAutenticado"]/tbody/tr[12]/td/table/tbody/tr/td''')

List the pages numbers

In [13]:
paginas = list(dict.fromkeys([beten.text.strip() for beten in numero_pagina if beten.text.strip().isdigit()]))

### Iterate each page and each row of the table to get the data

In [14]:
tabla_completa = [] # Empty list to store all data
for numero in paginas: # Iterate through each page number
    for boton in driver.find_elements(by=By.XPATH, value='''//*[@id="ctl00_ContentPlaceHolderContenido_gvListadoProcesosNoAutenticado"]/tbody/tr[12]/td/table/tbody/tr/td'''):
        if boton.text.strip() == numero: # Check if the button text matches the page number
            boton.click() # Click the button
            break # Break the loop after clicking the button
        
    time.sleep(2) # Wait for the page to load

    # Extract the table data
    filas = driver.find_elements(by=By.XPATH, value='''//*[@id="ctl00_ContentPlaceHolderContenido_gvListadoProcesosNoAutenticado"]/tbody''') # traer informacion de la tabla grande 
    
    for fila in filas: # Iterate through each row
        celdas = fila.find_elements(by=By.TAG_NAME, value='tr') # Find all table rows
        celdas = [celda for celda in celdas if not celda.get_attribute('class') and not celda.get_attribute('style')] # Filter out unwanted rows
        for celda in celdas: # Iterate through each cell
            datas = celda.find_elements(by=By.TAG_NAME, value='td') # Find all table data cells
            rows = [] # Initialize an empty list to store the row data
            for data in datas: # Iterate through each data cell
                rows.append(data.text) # Append the text of each data cell to the rows list
            tabla_completa.append(rows) # Append the row data to the complete table list

In [24]:
driver.quit() # Close the browser

### Create pandas Dataframe from extracted data

Convert list with extracted data to a pandas dataframe

In [15]:
df = pd.DataFrame(tabla_completa)

Assing a name for each column

In [16]:
df.columns = ['NO_PROCESO', 'EMPRESA', 'OBJETO_CONTRATACION', 'CATEGORIA', 'FECHA_APERTURA', 'HORA_APERTURA', 'FECHA_CIERRE', 'HORA_CIERRE']

Delete unnecessary columns

In [17]:
df.drop(columns=['HORA_CIERRE', 'HORA_APERTURA'], inplace=True)

Filter rows to get only the real data

In [18]:
df = df[df['NO_PROCESO'].str.lower().str.startswith('cm')]

Create a list with the keywords

In [19]:
key_words = ['caja', 'linux', 'servidor']

Filter data by the keywords

In [20]:
pattern = '|'.join(map(re.escape, key_words))  # Escapa caracteres especiales
df = df[df['OBJETO_CONTRATACION'].str.lower().str.contains(pattern, na=False)]

Create a column with the current date

In [21]:
df['FECHA_CONSULTA'] = datetime.now().strftime('%Y-%m-%d')

Organice columns

In [22]:
df = df.reindex(columns=['FECHA_CONSULTA', 'NO_PROCESO', 'EMPRESA', 'OBJETO_CONTRATACION', 'CATEGORIA', 'FECHA_APERTURA', 'FECHA_CIERRE'])

Export data to a CSV file

In [23]:
df.to_csv(f'''data/PROCESOS_EPM_{datetime.now().strftime('%Y-%m-%d')}.csv''', sep=';', index= False)